# Graph Convolutional Networks. See this [Paper][https://arxiv.org/abs/1609.02907], [Thomas Kipf's Github][https://github.com/tkipf]

In [1]:
import numpy as np
import scipy.sparse as sp
import os
import time
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA

import torch
import torch.optim as optim
import torch.nn as nn
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import torch.nn.functional as F

In [2]:
# settings
path = 'D:/Code/Graph/GCN/py_gcn/data/cora/'
dataset = 'cora'
n_hid = 16
n_class = 7
device = 'cuda' if torch.cuda.is_available else 'cpu'
seed = 42
epochs = 200
lr = 0.01
weight_decay = 5e-4
dropout = 0.5

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

## 1. Load Data  
* Adjacency matrix: $G \in R^{n\times n}$  
* Node features: $X \in R^{n\times p}$  
* Node labels: $t \in R^{n}$  
### 1.1 Utils

In [48]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    # CSR to COO
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

def build_graph(idx, edges):
    # node index range: 0 -> n-1
    # rename node index and edges
    idx_dict = {i:j for j,i in enumerate(idx)}
    n = len(idx_dict) # num_node
    re_edges = list(map(idx_dict.get, edges.reshape(-1)))
    re_edges = np.array(re_edges).reshape(edges.shape)
    # adjacency (sparse matrix)
    values = np.ones(re_edges.shape[0])
    v_i, v_j = re_edges[:,0], re_edges[:,1]
    G = sp.coo_matrix((values, (v_i, v_j)), shape=(n,n))
    # symmetric adjacency
    mask = (G + G.T)>=1
    G = mask*1. + sp.eye(mask.shape[0])
    # row normalization
    G = normalization(G)
    # numpy -> tensor sparse matrix
    G = sparse_mx_to_torch_sparse_tensor(G)
    return G

def class_name2labels(name):
    '''Convert class names to digits'''
    class_name = set(name)
    class_dict = {n:i for i,n in enumerate(class_name)}
    labels = torch.LongTensor(list(map(class_dict.get, name)))
    return labels

def load_data(path, dataset):
    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),\
        dtype=np.dtype(str))
    edges = np.genfromtxt("{}{}.cites".format(path, dataset),\
        dtype=np.int32) # shape: n_edges*2
    # node index
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    # Adjacency
    G = build_graph(idx, edges)
    # node features
    X = np.array(idx_features_labels[:, 1:-1], dtype=np.float32)
    X = torch.FloatTensor(normalization(X))
    # node labels
    t = class_name2labels(idx_features_labels[:, -1])
    t = torch.LongTensor(t)
    # train, val, test indexs
    idx_total = np.arange(t.shape[0])
    idx_train, idx_test = train_test_split(idx_total, test_size=0.6, stratify=t)
    idx_train, idx_val = train_test_split(idx_train, test_size=0.6, stratify=t[idx_train])
    idx_train, idx_none = train_test_split(idx_train, test_size=0.3, stratify=t[idx_train])
    idx_train_val_test = [idx_train, idx_val, idx_test]
    idx_train, idx_val, idx_test = map(torch.LongTensor,idx_train_val_test)

    return G,X,t,idx_train, idx_val, idx_test

def normalization(data, mode='l1'):
    norm = Normalizer(norm=mode)
    data = norm.fit_transform(data)
    return data

def accuracy(logits, labels):
    y = torch.argmax(logits, 1)
    num_corrects = (y==labels).sum()
    return num_corrects.float() / len(labels)

# train
def train(epoch):
    t = time.time()
    # forward
    model.train() # dropout
    optimizer.zero_grad()
    logits = model(X, G)
    loss_train = F.cross_entropy(logits[idx_train], ts[idx_train])
    acc_train = accuracy(logits[idx_train], ts[idx_train])
    # backward
    loss_train.backward()
    optimizer.step()

    # validation
    model.eval() # no dropout, no grad
    logits = model(X, G)

    loss_val = F.cross_entropy(logits[idx_val], ts[idx_val])
    acc_val = accuracy(logits[idx_val], ts[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))

# test
def test():
    model.eval()
    logits = model(X, G)
    loss_test = F.cross_entropy(logits[idx_test], ts[idx_test])
    acc_test = accuracy(logits[idx_test], ts[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

### 1.2 Load Data

In [41]:
G, X, ts, idx_train, idx_val, idx_test = load_data(path, dataset)

# use GPU
X = X.to(device)
G = G.to(device)
ts = ts.to(device)
idx_train = idx_train.to(device)
idx_val = idx_val.to(device)
idx_test = idx_test.to(device)

print(' Graph size: {}\nFeatures size: {}\nlabels: {}'\
    .format(G.size(), X.shape, t.shape))

Graph size: torch.Size([2708, 2708])
Features size: torch.Size([2708, 1433])
labels: torch.Size([2708])


## 2. Graph Convolutional Network Model  
$f = \sigma(GXW)$  
$G$: with self-loop

In [32]:
# model
class GCN(Module):
    def __init__(self, in_features, out_features, bias=True):
        super(GCN,self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        # Model parameters: weight and bias
        self.weight = Parameter(torch.FloatTensor(in_features,out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias',None)
        # Parameters initialization
        self.init_parameters()

    def init_parameters(self):
        nn.init.uniform_(self.weight.data)
        if self.bias is not None:
            self.bias.data.uniform_()

    def forward(self, x, adj):
        # Z = XW
        z = torch.mm(x, self.weight)
        # Z = AZ
        # adj is a sparse tensor: values, v_i, v_j
        z = torch.spmm(adj, z)
        if self.bias is not None:
            z = z+self.bias 
        return z

    def __repr__(self):
        info = self.__class__.__name__+\
            '('+str(self.in_features)+'->'+str(self.out_features)+')'
        return info

class GCN_Model(nn.Module):
    # Two layers
    def __init__(self, n_in, n_hid, n_class, dropout):
        super(GCN_Model, self).__init__()
        self.feat = GCN(n_in, n_hid)
        self.clf = GCN(n_hid, n_class)
        self.dropout = dropout
        
    def forward(self, x, adj):
        x = F.relu(self.feat(x,adj)) # 1st layer
        x = F.dropout(x, self.dropout, training=self.training)
        logits = self.clf(x,adj) # output
        return logits

In [33]:
# model
n_in, n_class = X.size(1), ts.max().item()+1

model = GCN_Model(n_in=n_in, n_hid=n_hid, n_class=n_class, dropout=dropout)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
model = model.to(device)

In [34]:
#

In [49]:
# Train model
t_total = time.time()
for epoch in range(epochs):
    train(epoch)
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Test model
test()

Epoch: 0001 loss_train: 2.2716 acc_train: 0.1485 loss_val: 2.0153 acc_val: 0.1308 time: 0.4198s
Epoch: 0002 loss_train: 2.0218 acc_train: 0.1716 loss_val: 1.9432 acc_val: 0.1308 time: 0.0179s
Epoch: 0003 loss_train: 2.0273 acc_train: 0.2244 loss_val: 1.8937 acc_val: 0.1615 time: 0.0175s
Epoch: 0004 loss_train: 2.0291 acc_train: 0.1716 loss_val: 1.8618 acc_val: 0.3015 time: 0.0160s
Epoch: 0005 loss_train: 1.9348 acc_train: 0.2442 loss_val: 1.8414 acc_val: 0.3015 time: 0.0160s
Epoch: 0006 loss_train: 1.9609 acc_train: 0.2244 loss_val: 1.8287 acc_val: 0.3015 time: 0.0150s
Epoch: 0007 loss_train: 1.9087 acc_train: 0.2706 loss_val: 1.8209 acc_val: 0.3015 time: 0.0160s
Epoch: 0008 loss_train: 1.9817 acc_train: 0.2574 loss_val: 1.8165 acc_val: 0.3015 time: 0.0150s
Epoch: 0009 loss_train: 1.8900 acc_train: 0.2706 loss_val: 1.8143 acc_val: 0.3015 time: 0.0160s
Epoch: 0010 loss_train: 1.8702 acc_train: 0.3036 loss_val: 1.8135 acc_val: 0.3015 time: 0.0150s
Epoch: 0011 loss_train: 1.8688 acc_train

## 3. Visualizing